In [1]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import SummaryIndex
from llama_index.core import Settings
from llama_index.core.tools import QueryEngineTool

from vllmpoint import get_vllm_embedder, get_vllm_llm

import logging
import sys

/home/akshaymambakam/genai/agentenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
llm = get_vllm_llm()
embed_model = get_vllm_embedder()

In [4]:
Settings.llm = llm
Settings.embed_model = embed_model

In [5]:
# Simple query by indexing
# TODO: Try multimedia inputs like mp3 and mp4
documents = SimpleDirectoryReader(
    input_files=["feedback.docx"]
).load_data()
feedback_index = VectorStoreIndex.from_documents(documents)

INFO:httpx:HTTP Request: POST http://g3.ai.qylis.com:7000/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://g3.ai.qylis.com:7000/v1/embeddings "HTTP/1.1 200 OK"


In [6]:
query = "What is the feedback about? Give me names."
query_engine = feedback_index.as_query_engine()
answer = query_engine.query(query)

INFO:httpx:HTTP Request: POST http://g3.ai.qylis.com:7000/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://g3.ai.qylis.com:7000/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://g3.ai.qylis.com:8000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://g3.ai.qylis.com:8000/v1/chat/completions "HTTP/1.1 200 OK"


In [7]:
print(answer.get_formatted_sources())
print("Query was:", query)
print("Answer was:", answer)

> Source (Doc id: 31ed660a-2ac8-46a0-be03-212e16053cc4): Interview Feedback Form - Qylis

Section 1: Candidate Details 

Name of Candidate   : ______ ____...
Query was: What is the feedback about? Give me names.
Answer was: The feedback is about an interview for a Data Science position. The names mentioned are Sohel and Akshay.


In [8]:
# Summarization
summary_index = SummaryIndex.from_documents(documents)

query_engine = summary_index.as_query_engine(response_mode="tree_summarize", streaming=True)
response = query_engine.query("<summarization_query>")
response.print_response_stream()
print('')

INFO:httpx:HTTP Request: POST http://g3.ai.qylis.com:8000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://g3.ai.qylis.com:8000/v1/chat/completions "HTTP/1.1 200 OK"
The candidate's strengths include being open and able to quickly grasp concepts. However, they require improvement in understanding the technical aspects of problems in their field.


In [9]:
documents = SimpleDirectoryReader(
    input_files=["paul_graham_essay.txt"]
).load_data()
essay_index = VectorStoreIndex.from_documents(documents)

# Define query engines and tools
tool1 = QueryEngineTool.from_defaults(
    query_engine=feedback_index.as_query_engine(),
    description="Use this query engine for feedback form.",
    # description="Use this query engine for an English essay.", Switch
)
tool2 = QueryEngineTool.from_defaults(
    query_engine=essay_index.as_query_engine(),
    description="Use this query engine for an English essay.",
    # description="Use this query engine for feedback form.", Switch
)

INFO:httpx:HTTP Request: POST http://g3.ai.qylis.com:7000/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://g3.ai.qylis.com:7000/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://g3.ai.qylis.com:7000/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://g3.ai.qylis.com:7000/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://g3.ai.qylis.com:7000/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://g3.ai.qylis.com:7000/v1/embeddings "HTTP/1.1 200 OK"


In [8]:
# Get as chat. Keeps track of conversation history
chat_engine = feedback_index.as_chat_engine()
chat1 = "Who is the interview candidate?"
response1 = chat_engine.chat(chat1)
print('Response1:', response1)
chat2 = "Who is the interviewer?"
response2 = chat_engine.chat(chat2)
print('Response2:', response2)

INFO:httpx:HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
Response1: Could you provide more context or details about the interview candidate you're referring to? Are you asking about a specific interview or a general concept?
INFO:httpx:HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
Response2: The interviewer is the person conducting the interview. They are typically responsible for asking questions, assessing the candidate's qualifications, and determining whether the candidate is a good fit 

In [13]:
chat3 = "Give me the summary of the feedback?"
response3 = chat_engine.chat(chat3)
print(response3)

INFO:httpx:HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://evaln-openai.op

In [10]:
# For interactive chat
chat_engine = feedback_index.as_chat_engine()
chat_engine.chat_repl()

===== Entering Chat REPL =====
Type "exit" to exit.

INFO:httpx:HTTP Request: POST http://g3.ai.qylis.com:8000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://g3.ai.qylis.com:8000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://g3.ai.qylis.com:7000/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://g3.ai.qylis.com:7000/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://g3.ai.qylis.com:8000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://g3.ai.qylis.com:8000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://g3.ai.qylis.com:8000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://g3.ai.qylis.com:8000/v1/chat/completions "HTTP/1.1 200 OK"
Assistant: The summary of the feedback is that the candidate, Sohel, has potential but needs to work on their technical skills.

INFO:httpx:HTTP Request: POST http://g3.ai.qylis.com:8000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Reque